In [1]:
! pip install -U git+https://github.com/Deci-AI/super-gradients

  Cloning https://github.com/Deci-AI/super-gradients to /tmp/pip-req-build-_t23gws_
  Running command git clone --filter=blob:none --quiet https://github.com/Deci-AI/super-gradients /tmp/pip-req-build-_t23gws_
  Resolved https://github.com/Deci-AI/super-gradients to commit e0ccacf8868ffa1296fa4f8407c03d2bc227312c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
! sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /usr/local/lib/python3.11/dist-packages/super_gradients/training/pretrained_models.py
! sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /usr/local/lib/python3.11/dist-packages/super_gradients/training/utils/checkpoint_utils.py

In [3]:
from super_gradients.common.object_names import Models
from super_gradients.conversion import DetectionOutputFormatMode
from super_gradients.training import models

model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")

The console stream is logged into /root/sg_logs/console.log


[2024-12-14 08:54:58] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2024-12-14 08:55:05] INFO - utils.py - NumExpr defaulting to 2 threads.
[2024-12-14 08:55:07] INFO - env_sanity_check.py - Library check is not supported when super_gradients installed through "git+https://github.com/..." command
[2024-12-14 08:55:08] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2024-12-14 08:55:08] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions 

In [6]:
# export the model for compatibility with Frigate

model.export("yolo_nas_s.onnx",
             output_predictions_format=DetectionOutputFormatMode.FLAT_FORMAT,
             max_predictions_per_image=20,
             confidence_threshold=0.4,
             input_image_shape=(320,320),
            )

Model exported successfully to yolo_nas_s.onnx
Model expects input image of shape [1, 3, 320, 320]
Input image dtype is torch.uint8
Exported model already contains preprocessing (normalization) step, so you don't need to do it manually.
Preprocessing steps to be applied to input image are:
Sequential(
  (0): CastTensorTo(dtype=torch.float32)
  (1): ApplyMeanStd(mean=[0.], scale=[255.])
)

Exported model contains postprocessing (NMS) step with the following parameters:
    num_pre_nms_predictions=1000
    max_predictions_per_image=20
    nms_threshold=0.7
    confidence_threshold=0.4
    output_predictions_format=flat

Exported model is in ONNX format and can be used with ONNXRuntime
To run inference with ONNXRuntime, please use the following code snippet:

    import onnxruntime
    import numpy as np
    session = onnxruntime.InferenceSession("yolo_nas_s.onnx", providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
    inputs = [o.name for o in session.get_inputs()]
    outputs 

In [7]:
from google.colab import files

files.download('yolo_nas_s.onnx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>